In [1]:
#Scraping libraries
%pip install beautifulsoup4
#Open-source LLM for NER extraction and setting up LLM-libraries 
%pip install gliner

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timezone

# Removing HTML tags from text
def clean_html(text):
    if not text:
        return None
    return BeautifulSoup(text, "html.parser").get_text(separator=" ", strip=True)
    
# Paths to RSS feed XML files
paths =['/kaggle/input/feed-base/recursive_rss_feed','/kaggle/input/feed-base/uktech_rss_feed','/kaggle/input/feed-base/startupitalia_rss_feed']

rss_feed_data = pd.DataFrame()
for path in paths :
    with open(path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "xml")
    
    items = []
    source = soup.title.get_text(strip=True)
    # Extracting data from each RSS item
    for item in soup.find_all("item"):
        guid = item.find("guid").text if item.find("guid") else None
        link = item.find("link").text if item.find("link") else None
        title = item.find("title").text if item.find("title") else None
        description = item.find("description").text if item.find("description") else None
        pub_date = item.find("pubDate").text if item.find("pubDate") else None
        categories = [cat.text for cat in item.find_all("category")]
    
        items.append({
            "guid": guid,
            "link": link,
            "published_at": pub_date,
            "title": title,
            "description": description,
            "categories": "|".join(categories) if categories else None,
            "source": source,
            "fetched_at": datetime.now(timezone.utc).isoformat()
        })
    
    df = pd.DataFrame(items)
    
    df["description"] = df["description"].apply(clean_html)

    rss_feed_data = pd.concat([rss_feed_data, df], ignore_index=True)
    

In [8]:
rss_feed_data.to_csv("/kaggle/working/rss_feed_data.csv", index=False)
rss_feed_data.head()

,guid,link,published_at,title,description,categories,source,fetched_at
0,https://therecursive.com/?p=43597,https://therecursive.com/cee-startup-tech-week...,"Fri, 09 Jan 2026 11:29:59 +0000",CEE Startup & Tech Weekly: CEE-Rooted Companie...,The Recursive’s weekly roundup aims to cover k...,Deals|News,TheRecursive.com,2026-01-16T20:02:26.529004+00:00
1,https://therecursive.com/?p=43472,https://therecursive.com/cleantech-for-see-eu-...,"Fri, 09 Jan 2026 10:12:27 +0000",Regional Cleantech Is Done Waiting.,The cleantech narrative in Europe has long bee...,Analysis|CleanTech|Ecosystem,TheRecursive.com,2026-01-16T20:02:26.529183+00:00
2,https://therecursive.com/?p=43419,https://therecursive.com/compliance-changes-fi...,"Thu, 08 Jan 2026 14:42:49 +0000",What Compliance Changes Are Coming to FinTech ...,"If you work in fintech, you’ve no doubt often ...",Fintech|Guest Article|Legal|DORA|ESMA|mica,TheRecursive.com,2026-01-16T20:02:26.529355+00:00
3,https://therecursive.com/?p=43559,https://therecursive.com/cee-founders-stop-hir...,"Wed, 07 Jan 2026 10:41:01 +0000","CEE Founders, Stop Hiring “Fundraising Consult...",I’ve noticed a new trend in the CEE ecosystem:...,Founder's School|Investors|Startups|fundraising,TheRecursive.com,2026-01-16T20:02:26.529512+00:00
4,https://therecursive.com/?p=43563,https://therecursive.com/cee-robotics-ai-start...,"Tue, 06 Jan 2026 15:32:35 +0000",10 CEE-Founded Robotics Startups You Should Know,The robotics and AI robotics market in Eastern...,Deep Tech|Listicle,TheRecursive.com,2026-01-16T20:02:26.529663+00:00


In [16]:
import re
import time
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"
        "AppleWebKit/537.36 (KHTML, like Gecko)"
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
}


def _clean(s):
    return re.sub(r"\s+", " ", (s or "")).strip()

def _pick_content_root(soup):
    # UKTech
    uktech = soup.select_one("div.js-post-content")
    if uktech:
        return uktech

    # The Recursive
    entry = soup.select_one("div.entry-main-content")
    if entry:
        bialty = entry.select_one("div.bialty-container")
        return bialty or entry

    # StartupItalia
    return (
        soup.select_one("div.post-content.cf.entry-content.content-spacious")
        or soup.select_one("div.post-content-wrap")
    )

#Fetching content from the url and cleaning the text
def scrape_and_clean_text(url):
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    for t in soup(["script", "style", "noscript"]):
        t.decompose()
        
    root = _pick_content_root(soup)

    blocks = []
    if root:
        for el in root.select("h1,h2,h3,p,li"):
            txt = _clean(el.get_text(" ", strip=True))
            if not txt:
                continue
            if txt.lower().startswith(("subscribe", "sign up", "related:", "share this")):
                continue
            blocks.append(txt)

    return "\n\n".join(blocks)



results = []
skipped_sources = set()

for source, group in rss_feed_data.groupby("source"):

    if source in skipped_sources:
        continue

    print(f"Processing source: {source}")

    for _, row in group.iterrows():
        url = row.get("link")
        if not url:
            continue

        try:
            article_text = scrape_and_clean_text(url)

            results.append({
                "guid": row.get("guid"),
                "url": row.get("link"),
                "source": source,
                "published_at": row.get("published_at"),
                "title": row.get("title"),
                "description": row.get("description"),
                "categories": row.get("categories"),
                "fetched_at": row.get("fetched_at"),
                "text": article_text,
                "text_char_len": len(article_text),
            })

            time.sleep(0.5)

        except Exception as e: 
            print(f"Skipping source '{source}' due to error: {e}") 
            skipped_sources.add(source) 
            break

print("Done")
print(f"Sources total: {rss_feed_data['source'].nunique()}")
print(f"Sources skipped : {len(skipped_sources)}")

results_df = pd.DataFrame(results)

Processing source: StartupItalia
Skipping source 'StartupItalia' due to error: 403 Client Error: Forbidden for url: https://startupitalia.eu/economy/andreessen-horowitz-nuovo-fondo-record/
Processing source: TheRecursive.com
Processing source: UKTN
Done
Sources total: 3
Sources skipped : 1


In [17]:
results_df.head()

,guid,url,source,published_at,title,description,categories,fetched_at,text,text_char_len
0,https://therecursive.com/?p=43597,https://therecursive.com/cee-startup-tech-week...,TheRecursive.com,"Fri, 09 Jan 2026 11:29:59 +0000",CEE Startup & Tech Weekly: CEE-Rooted Companie...,The Recursive’s weekly roundup aims to cover k...,Deals|News,2026-01-16T20:02:26.529004+00:00,The Recursive’s weekly roundup aims to cover k...,2418
1,https://therecursive.com/?p=43472,https://therecursive.com/cleantech-for-see-eu-...,TheRecursive.com,"Fri, 09 Jan 2026 10:12:27 +0000",Regional Cleantech Is Done Waiting.,The cleantech narrative in Europe has long bee...,Analysis|CleanTech|Ecosystem,2026-01-16T20:02:26.529183+00:00,The cleantech narrative in Europe has long bee...,6387
2,https://therecursive.com/?p=43419,https://therecursive.com/compliance-changes-fi...,TheRecursive.com,"Thu, 08 Jan 2026 14:42:49 +0000",What Compliance Changes Are Coming to FinTech ...,"If you work in fintech, you’ve no doubt often ...",Fintech|Guest Article|Legal|DORA|ESMA|mica,2026-01-16T20:02:26.529355+00:00,"If you work in fintech, you’ve no doubt often ...",7412
3,https://therecursive.com/?p=43559,https://therecursive.com/cee-founders-stop-hir...,TheRecursive.com,"Wed, 07 Jan 2026 10:41:01 +0000","CEE Founders, Stop Hiring “Fundraising Consult...",I’ve noticed a new trend in the CEE ecosystem:...,Founder's School|Investors|Startups|fundraising,2026-01-16T20:02:26.529512+00:00,I’ve noticed a new trend in the CEE ecosystem:...,5200
4,https://therecursive.com/?p=43563,https://therecursive.com/cee-robotics-ai-start...,TheRecursive.com,"Tue, 06 Jan 2026 15:32:35 +0000",10 CEE-Founded Robotics Startups You Should Know,The robotics and AI robotics market in Eastern...,Deep Tech|Listicle,2026-01-16T20:02:26.529663+00:00,The robotics and AI robotics market in Eastern...,5459


In [18]:
from gliner import GLiNER
import warnings
#Loading the model 
warnings.filterwarnings(
    "ignore",
    message="Sentence of length .* "
)
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
# Entity labels to extract
LABELS = ["Startup", "Investor", "Funding amount", "Funding stage", "Founder"]


def extract_gliner_fields(text):
    if not text:
        return {
            "startup_name": None,
            "founders": None,
            "funding_amount": None,
            "funding_stage": None,
            "investors": None,
            "extraction_confidence": "low",
        }

    ents = model.predict_entities(text, LABELS)
    #Picking highest-confidence entity for a label
    def pick(label):
        items = [e for e in ents if e["label"] == label]
        items.sort(key=lambda x: x.get("score", 0), reverse=True)
        return items[0]["text"] if items else None

    def pick_many(label):
        seen, out = set(), []
        for e in ents:
            if e["label"] == label and e["text"] not in seen:
                seen.add(e["text"])
                out.append(e["text"])
        return " | ".join(out) if out else None

    startup = pick("Startup")
    amount = pick("Funding amount")
    stage = pick("Funding stage")
    founders = pick_many("Founder")
    investors = pick_many("Investor")

    if startup and amount and investors:
        confidence = "high"
    elif startup and (amount or investors):
        confidence = "medium"
    elif startup:
        confidence = "low"
    else:
        confidence = "low"

    return {
        "startup_name": startup,
        "founders": founders,
        "funding_amount": amount,
        "funding_stage": stage,
        "investors": investors,
        "extraction_confidence": confidence,
    }


In [21]:
gliner_cols = results_df["text"].apply(extract_gliner_fields)
gliner_df = pd.DataFrame(gliner_cols.tolist())

results_df = pd.concat(
    [results_df.reset_index(drop=True), gliner_df],
    axis=1
)

results_df.head()
results_df = results_df.fillna("NA")
results_df.to_csv("/kaggle/working/output.csv", index=False)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
from transformers import pipeline

# Loading zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Classification label sets
INDUSTRIES = ["Fintech", "Healthtech", "AI", "Robotics",
    "SaaS", "Climate", "E-commerce", "Biotech", "Other"]

EVENT_TYPES = [ "funding", "acquisition", "product_launch",
    "partnership", "market_report", "regulation", "other"]

def classify_article(text):
    if not text or len(text) < 50:
        return {
            "industry_category": "NA",
            "key_event_type": "NA",
        }
    text = text[:5000]
    industry_result = classifier(text, INDUSTRIES)
    event_result = classifier(text, EVENT_TYPES)

    return {
        "ai_industry_category": industry_result["labels"][0],
        "ai_key_event_type": event_result["labels"][0],
    }

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [23]:
import pandas as pd

cls_rows = results_df["text"].apply(classify_article)
cls_df = pd.DataFrame(cls_rows.tolist())

results_df = pd.concat(
    [results_df.reset_index(drop=True), cls_df],
    axis=1
)

results_df.to_csv("/kaggle/working/output_with_AI.csv", index=False)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
